# [실습1] LLM와 Langchain으로 데이터 분석 코드 실행하기

## 실습 목표
---
Langchain과 LLM을 결합해서 주어진 데이터를 분석하는 파이썬 코드를 생성하고, 이를 Langchain을 활용해 실행하는 방법을 학습합니다.

## 실습 목차
---

1. **데이터 특징 추출:** 주어진 데이터가 어떻게 구성되어 있는지 추출하고 이를 시스템 프롬프트에 적용합니다.

2. **데이터 분석 체인 구성:** 데이터를 분석하는 코드를 생성 및 실행하고, 그 결과를 출력하는 체인을 구성합니다.

## 실습 개요
---
LangChain을 활용해서 자연어로 데이터를 분석할 수 있는 챗봇을 구현하고 사용해봅니다.

## 0. 환경 설정
- 필요한 라이브러리를 불러옵니다.

In [3]:
import contextlib
import io
import os

import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_experimental.tools.python.tool import PythonAstREPLTool

- Ollama를 통해 Mistral 7B 모델을 불러옵니다. 처음 불러오는데는 다운로드 시간 포함 약 3분이 소요됩니다.

In [ ]:
!ollama pull mistral:7b

## 1. 데이터 특징 추출
- 주어진 데이터가 어떻게 구성되어 있는지 추출하고 이를 시스템 프롬프트에 적용합니다.
- 이번 실습에 사용할 데이터는 2일차 실습 중 **Langchain을 이용한 머신러닝 기반 데이터 분석 실무 프로젝트** 에 사용하는 데이터입니다.

먼저, mistral:7b 모델을 사용하는 ChatOllama 객체와 OllamaEmbeddings 객체를 생성합니다.

In [5]:
llm = ChatOllama(model="mistral:7b")
embeddings = OllamaEmbeddings(model="mistral:7b")

In [ ]:
# OPEN AI API 사용 시
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
OPENAI_API_KEY= 'sk'
llm_openai = ChatOpenAI(model='gpt-4o-mini', openai_api_key=OPENAI_API_KEY)
embeddings_openai = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


다음으로, 데이터를 불러오고, 데이터의 컬럼명을 변수에 저장합니다.

대부분의 경우 컬럼명에서 데이터의 특성을 파악할 수 있기 때문에, LLM이 사용자의 질문에 맞는 데이터를 어떤 코드를 통해 추출할지 추론하는 좋은 단서가 됩니다.

In [ ]:
# 데이터를 불러오고, 이름과 컬럼명을 저장합니다.
data_dir = './data'
df_inkjet = pd.read_csv(os.path.join(data_dir, 'InkjetDB_preprocessing.csv'), index_col=0)

# 데이터를 저장한 변수명을 LLM에 제공하여 이 변수를 활용하는 코드를 작성하게 할 수 있습니다.
df_name = "df_inkjet"
df_columns = ", ".join(df_inkjet.columns)

In [ ]:
# df_columns 확인하기
print(df_columns)

다음으로, 데이터의 컬럼명을 바탕으로 코드를 쿼리하는 프롬프트를 작성합니다.

In [7]:
system_message = "당신은 주어진 데이터를 분석하는 데이터 분석가입니다.\n"
system_message += f"주어진 DataFrame에서 데이터를 출력하여 주어진 질문에 답할 수 있는 파이썬 코드를 작성하세요. {df_name} DataFrame에는 액세스할 수 있습니다.\n"
system_message += f"`{df_name}` DataFrame에는 다음과 같은 열이 있습니다: {df_columns}\n"
system_message += "데이터는 이미 로드되어 있으므로 데이터 로드 코드를 생략해야 합니다."

message_with_data_info = [
    ("system", system_message),
    ("human", "{question}"),
]

시스템 프롬프트를 확인해 봅시다.

In [ ]:
print(system_message)

이전 실습에서 구성한 체인과 비슷하게, 코드를 생성하는 체인을 구성하고 실행해 봅시다.

In [9]:
prompt_with_data_info = ChatPromptTemplate.from_messages(message_with_data_info)

# 체인 구성
code_gen_chain = (
    {"question": RunnablePassthrough()}
    | prompt_with_data_info
    | llm
    | StrOutputParser()
)

In [ ]:
print(code_gen_chain.invoke("Velocity가 가장 큰 데이터를 찾아줘"))

그럴듯한 코드를 생성했습니다. 이제 이 코드를 복사해서 실행해보세요.
- 앞선 코드에서 `df_inkjet` 데이터를 불러왔기 때문에, 생성된 코드를 그대로 복사-붙여넣기 한 후 실행해주시면 됩니다.

In [ ]:
# 체인의 답변 중 코드 부분만 추출해서 복사 붙여넣기 해주세요.
# 직접해보시고 안되시면 아래 코드를 참조해주세요:)
# max_velocity_row = df_inkjet.loc[df_inkjet['Velocity'].idxmax()]
# print(max_velocity_row)

목적에 맞게 잘 데이터를 추출한 것을 확인할 수 있습니다. 

이제 이 코드를 수동으로 실행하지 않고, 체인을 통해 실행하고 그 결과만 바로 받아볼 수 있도록 데이터 분석 체인을 구성해봅시다.

## 2. 데이터 분석 체인 구성
- 데이터를 분석하는 코드를 생성하고, 코드를 실행하고, 그 결과를 답변하는 체인을 구성합니다.

조금 전 저희는 코드 생성 체인이 생성한 코드를 수동으로 복사하여 코드 블럭에 입력한 후 결과를 확인했습니다.

이 과정을 자동화 하여 자연어로 원하는 데이터를 입력하면, 그에 맞춘 데이터만 나오는 체인을 구성해 봅시다.

### 2-1. 코드 복사
먼저, LLM이 생성한 코드를 찾아서 복사하는 기능을 추가해 봅시다.

입력된 텍스트에서 파이썬 코드만 추출하는 함수를 정의합니다.

In [12]:
def python_code_parser(input: str) -> str:
    # LLM은 대부분 ``` 블럭 안에 코드를 출력합니다. 이를 활용합니다.
    # ```python (코드) ```, 혹은 ``` (코드) ``` 형태로 출력됩니다. 두 경우 모두에 대응하도록 코드를 작성합니다.
    processed_input = input.replace("```python", "```").strip() 
    parsed_input_list = processed_input.split("```")

    # 만약 ``` 블럭이 없다면, 입력 텍스트 전체가 코드라고 간주합니다.
    # 아닐 경우 이어지는 코드 실행 과정에서 예외 처리를 통해 오류를 확인할 수 있습니다.
    if len(parsed_input_list) == 1:
        return processed_input

    # 코드 부분만 추출합니다. 
    # LLM은 여러 코드 블럭에 걸쳐 필요한 코드를 출력할 수 있으므로, 코드가 있는 홀수 번째 텍스트를 모두 저장합니다.
    parsed_code_list = []
    for i in range(1, len(parsed_input_list), 2):
        parsed_code_list.append(parsed_input_list[i])
    
    # 코드 부분을 하나로 합칩니다.
    return "\n".join(parsed_code_list)

1. input.replace("```python", "```")
이 부분은 **입력된 문자열에서 ```python이라는 패턴을 찾아서 ```로 바꿉니다.

2. .strip()
이 부분은 문자열의 앞뒤에 있는 공백이나 줄바꿈 문자를 모두 제거합니다.

3. processed_input.split("```")
``` 기준으로 문자열을 나눕니다.

4. if len(parsed_input_list) == 1:
    return processed_input
parsed_input_list에 코드가 없을 경우 len(parsed_input_list)는 1입니다.

5. parsed_code_list = []
   for i in range(1, len(parsed_input_list), 2):
        parsed_code_list.append(parsed_input_list[i])
반복문을 통해 parsed_code_list에 코드 부분만 넣기

6. "\n".join(parsed_code_list)
리스트에 있는 요소들을 각각 줄바꿈하여 연결
EX) A = [1,2,3]
1
2
3


코드 분해해보기 - 1

In [ ]:
def python_code_parser1(input: str) -> str:
    # LLM은 대부분 ``` 블럭 안에 코드를 출력합니다. 이를 활용합니다.
    # ```python (코드) ```, 혹은 ``` (코드) ``` 형태로 출력됩니다. 두 경우 모두에 대응하도록 코드를 작성합니다.
    processed_input = input.replace("```python", "```").strip() 
    parsed_input_list = processed_input.split("```")
    print("\n\nprocessed_input : \n\n", processed_input)
    print("\n\nparsed_input_list : \n\n", parsed_input_list)
    print("\n\nlen(parsed_input_list) : \n\n", len(parsed_input_list))


    # 만약 ``` 블럭이 없다면, 입력 텍스트 전체가 코드라고 간주합니다.
    # 아닐 경우 이어지는 코드 실행 과정에서 예외 처리를 통해 오류를 확인할 수 있습니다.
    if len(parsed_input_list) == 1:
        return processed_input

    # 코드 부분만 추출합니다. 
    # LLM은 여러 코드 블럭에 걸쳐 필요한 코드를 출력할 수 있으므로, 코드가 있는 홀수 번째 텍스트를 모두 저장합니다.
    parsed_code_list = []
    for i in range(1, len(parsed_input_list), 2):
        parsed_code_list.append(parsed_input_list[i])
    
    # 코드 부분을 하나로 합칩니다.
    return "\n".join(parsed_code_list)

python_code_parser1(code_gen_chain.invoke("Velocity가 가장 큰 데이터를 찾아줘"))

이제 함수를 체인에 연결합니다.
- 단순 함수를 체인에 연결하면, LangChain은 암묵적으로 "Runnable"한 `RunnableLambda` 로 변환해서 체인에 연결합니다.

In [13]:
code_gen_chain_with_parser = (
    code_gen_chain
    | python_code_parser
)

아까랑 같은 질문을 해봅시다.

In [ ]:
print(code_gen_chain_with_parser.invoke("Velocity가 가장 큰 데이터를 찾아줘"))

생성하는 코드는 비슷하지만, 앞뒤로 설명이 없고 코드 부분만 깔끔하게 출력될 것입니다. 만약 그렇지 않을 경우, 다시 한번 실행해주세요.

### 2-2. 코드 실행
- 코드만 깔끔하게 출력했으니, 이제 이 코드를 실행해봅시다.

파이썬에는 코드 텍스트를 실행하는 `exec()` 함수가 있습니다. 이 함수를 활용하여 입력된 코드를 실행하고, 그 출력 결과를 반환하는 함수를 정의합니다.

In [15]:
def run_code(input_code: str):
    # 코드가 출력한 값을 캡쳐하기 위한 StringIO 객체를 생성합니다.
    output = io.StringIO()
    try:
        # Redirect stdout to the StringIO object
        with contextlib.redirect_stdout(output):
            # Python 3.10 버전이므로, 키워드 인자를 사용할 수 없습니다.
            # 코드가 실행하면서 출력한 모든 결과를 캡쳐합니다.
            exec(input_code, {"df_inkjet": df_inkjet})
    except Exception as e:
        # 에러가 발생할 경우, 이를 StringIO 객체에 저장합니다.
        print(f"Error: {e}", file=output)
    # StringIO 객체에 저장된 값을 반환합니다.

    return output.getvalue()

   

1. io.StringIO()
문자열을 메모리에 파일처럼 저장할 수 있는 객체를 생성합니다. 이후 표준 출력(print)으로 나오는 내용을 이 StringIO 객체에 저장합니다.

2. contextlib.redirect_stdout(output)
파이썬의 표준 출력(stdout)을 output이라는 StringIO 객체로 리다이렉트합니다. 
이 블록 안에서 print 함수가 호출되면, 콘솔에 출력되지 않고 output에 기록됩니다.

3. exec(input_code, {"df_inkjet": df_inkjet})
exec 함수는 문자열로 주어진 코드를 파이썬 인터프리터에서 실행합니다. 
여기서는 input_code 문자열을 실행하면서, 해당 코드가 사용할 수 있는 전역 변수로 df_inkjet이라는 데이터프레임을 전달합니다.

4. except Exception as e:
만약 exec로 실행된 코드에 에러가 발생하면 예외를 잡아서 처리합니다.

5. print(f"Error: {e}", file=output)
에러가 발생할 경우, 에러 메시지를 문자열로 저장된 output에 기록합니다. file=output을 사용해 output 객체에 기록합니다.

6. output.getvalue()
getvalue() 메소드를 사용하면, 이 스트림에 저장된 모든 문자열을 반환할 수 있습니다.

아까와 동일하게, 함수를 체인에 연결합니다.
- 단순 함수를 체인에 연결하면, LangChain은 암묵적으로 "Runnable"한 `RunnableLambda` 로 변환해서 체인에 연결합니다.

In [16]:
code_execute_chain = (
    code_gen_chain_with_parser |
    run_code
)

아까랑 같은 질문을 해봅시다.

In [ ]:
print(code_execute_chain.invoke("Velocity가 가장 큰 데이터를 찾아줘"))

이번에는 코드 실행 결과만 반환하는 것을 확인할 수 있습니다.

만약 위와 같이 `stdout`을 캡쳐하는 것이 불가능하다면, Langchain-experimental의 `PythonAstREPLTool` 을 활용할 수도 있습니다.

단, `PythonAstREPLTool`은 코드가 가장 마지막에 출력한 출력 값만 반환합니다.

In [18]:
repl_execute_chain = (
    code_gen_chain_with_parser |
    PythonAstREPLTool(globals={"df_inkjet": df_inkjet})
)

In [ ]:
print(repl_execute_chain.invoke("Velocity가 가장 큰 데이터를 찾아줘"))

저희는 원하는 데이터를 자연어로 입력하면, 이를 바탕으로 자동으로 코드를 생성하고 결과를 반환하는 체인을 구성했습니다. 

그러나, 현재 구성한 체인은 특정 데이터를 찾아달라는 질문 뿐만 아니라, 이를 설명해달라는 질문에도 데이터만 반환합니다. 

In [ ]:
print(code_execute_chain.invoke("Velocity가 가장 큰 데이터에 대해 설명해줘"))

### 추가 실습
- 저희는 저 문제를 4챕터에서 해결해 볼 것입니다.
- 그 전에, `code_execute_chain` 체인과 다른 구성 요소를 활용해서 이 문제를 해결한 챗봇을 구현해보세요.

In [ ]:
# Hint. code_execute_chain이 반환한 결과를 Context로 받는 또 다른 체인을 만들어 보세요

In [21]:
#예시
messages_with_variables = [
    ("system", "이름과 값에 대해 설명해주는 전문가입니다."),
    ("human", "{run_code}의 내용을 이름과 값에 대해 설명해줘 ."),
]
prompt = ChatPromptTemplate.from_messages(messages_with_variables)
parser = StrOutputParser()


context_chain = (
    code_gen_chain_with_parser |
    run_code
    |prompt | llm | parser
)

print(context_chain.invoke("Velocity가 가장 큰 데이터에 대해 설명해줘"))